In [5]:
from IPython.display import display, clear_output, HTML, Image
import time
import random
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [6]:
import requests
from bs4 import BeautifulSoup
import json
import time

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok
  

In [7]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [8]:
filenames = ["b10_09.png", "y09_10.png", "b14_10.png", "y10_14.png","b14_12.png", "y12_14.png","b14_11.png", "y11_14.png","b16_12.png", "y12_16.png","b18_13.png", "y13_18.png","b18_14.png", "y14_18.png","b18_16.png", "y16_18.png","b20_15.png", "y15_20.png","b20_18.png", "y18_20.png","b21_18.png", "y18_21.png", "b12_09.png", "y09_12.png"]

In [39]:
data_consent_info = """DATA CONSENT INFORMATION:

Please read:

we wish to record your response data to an anonymised public data repository.

Your data will be used for educational teaching purposes practising data analysis and visualisation.

Please type yes in the box below if you consent to the upload."""

print(data_consent_info)

result = input("> ")

if result.lower() == "yes":

    print("Thanks for your participation.")

    print("Please contact philip.lewis@ucl.ac.uk")

    print("If you have any questions or concerns")

    print("regarding the stored results.")

else:

# end code execution by raising an exception

    raise(Exception("User did not consent to continue test."))

DATA CONSENT INFORMATION:

Please read:

we wish to record your response data to an anonymised public data repository.

Your data will be used for educational teaching purposes practising data analysis and visualisation.

Please type yes in the box below if you consent to the upload.


>  YES


Thanks for your participation.
Please contact philip.lewis@ucl.ac.uk
If you have any questions or concerns
regarding the stored results.


In [41]:
id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend

- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise

then your unique identifer would be CBTC

"""

print(id_instructions)

user_id = input("> ")

print("User entered id:", user_id)



Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend

- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise

then your unique identifer would be CBTC




>  VRTV


User entered id: VRTV


In [42]:
from IPython.display import display, clear_output, HTML, Image
 
top_area = widgets.Output(layout={"height":"60px"})
main_area = widgets.Output(layout={"height":"250px"})
bottom_area = widgets.Output(layout={"height":"60px"})

newdata = {
'id':"A",
'age': 0,
'score': 0,
}

test_df = {
    'filename': [],
    'nL': [],
    'nR': [],
    'corr_ans': [],
    'ans': [],
    'score': []
}

btn1 = widgets.Button(description="Y")
btn2 = widgets.Button(description="B")

btn1.on_click(register_btn_event)
btn2.on_click(register_btn_event)

panel = widgets.HBox([btn1, btn2])
top_area.append_display_data( HTML("<h1>ANS Test</h1>") )

bottom_area.append_display_data(panel)

display(top_area)
display(main_area)
display(bottom_area)

n = len(filenames)
score = 0
newdata["id"] = user_id
ans_age = input("What is your age?")
newdata["age"] = ans_age

for i in range(n):
    picture = Image(filenames[i], width = 400)
    with main_area: display(picture)
    with main_area: time.sleep(0.75)
    with main_area: clear_output(wait = False)
    with main_area: print("Which colour had more dots?")
    result = wait_for_event(timeout=3)
    ans = 0
    if result['description'] == '':
        with main_area: display("Too slow!")
    elif result['description'].lower() ==  filenames[i][0]:
        with main_area: display("Correct!")
        ans = 1
        score = score + 1
    else:
        with main_area: display("Wrong!")

    test_df['filename'].append(filenames[i])
    test_df['nL'].append(filenames[i][1:3])
    test_df['nR'].append(filenames[i][4:6])
    test_df['corr_ans'].append(filenames[i][0])
    test_df['ans'].append(result['description'].lower())
    test_df['score'].append(ans)
    wait_for_event(timeout=1)
    with main_area: clear_output()
        
with bottom_area: clear_output()

with main_area: print(f"You scored {score}")
newdata["score"] = score

Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython…

Output(layout=Layout(height='250px'))

Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chi…

What is your age? 21


In [43]:
Question_info = pd.DataFrame(test_df)
newdata["results_json"] = Question_info.to_json()
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSe5A2E0XojVHl2GEunHO2Dia91HkXRvr90CHe6DgLFa7iBN8Q/viewform?usp=sf_link"
send_to_google_form(newdata, form_url)

True

True